# 开了解LCEL
LCEL是LangChain帮助我们快速构建复杂的链式组件的基本语法，并且流处理，并且调用和日志处理都是可以开箱即用的。

## 基本案例开始（prompt+context+output parser）

(动态提示词模板+输出) 这个是最常见也是最基本调的用方式，下面我将用获得城市旅游信息作为案例

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import get_openai_callback

# 提示词模板
prompt = ChatPromptTemplate.from_template("我想去{topic}旅行，我想知道这个地方有什么好玩的")

# gpt模型
model = ChatOpenAI(
    openai_api_key="",
    openai_api_base="",
    temperature=.7
                 )

# 输出模板
print("输出模板:",prompt)

# 输出解析器
output_parser = StrOutputParser()

chain = prompt | model | output_parser

result = chain.invoke({"topic": "云南"})

print("结果:",result)


输出模板: input_variables=['topic'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='我想去{topic}旅行，我想知道这个地方有什么好玩的'))]
结果: 云南是一个充满魅力的地方，有许多值得一游的景点和活动。以下是一些你可能会感兴趣的云南旅行目的地和活动：

1. 石林：世界自然遗产，以其奇特的石灰岩地貌和壮丽的景色而闻名。

2. 丽江古城：保存完好的古城，有着悠久的历史和文化底蕴，是世界文化遗产。

3. 香格里拉：位于横断山脉腹地，有着壮丽的自然风光和丰富的藏传佛教文化。

4. 云南民族村寨：可以体验不同民族的风俗文化和传统手工艺品。

5. 滇池：中国第六大淡水湖，景色优美，可以游船观景，还有丰富的水产资源。

6. 云南美食：尝试当地的特色美食，如过桥米线、火锅、酸菜鱼等。

除此之外，云南还有许多户外活动可供选择，如徒步旅行、骑行、登山等。希望你在云南旅行中能有一个愉快的体验！


注意下面这个代码，它就是LCEL语法将不同的组件拼凑成一个链

In [16]:
chain = prompt | model | output_parser

**|** 符号类似于Unix管道操作符，它将不同组件链接在一起，将一个组件的输出作为下一个组件的输入。
在这个链条中，用户输入被传递到提示模板，然后提示模板的输出被传递到模型，接着模型的输出被传递到输出解析器。让我们分别看看每个组件，真正理解正在发生的事情。

### 1.Prompt
**prompt**是一个基础prompt模板，它能接受一个变量  **{topic}**，并生成一个最终**prompt_value**。**prompt_value**是一个包装完成的提示的对象，可以传递给LLM（接受字符串作为输入）或ChatModel（接受消息序列作为输入）。它可以与任何一种语言模型类型一起使用，因为它定义了生成**BaseMessages**和生成字符串的逻辑。

In [17]:
# promt模板与参数结合
prompt_value = prompt.invoke({"topic":"云南"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='我想去云南旅行，我想知道这个地方有什么好玩的')])

In [18]:
prompt_value.to_messages()

[HumanMessage(content='我想去云南旅行，我想知道这个地方有什么好玩的')]

In [19]:
prompt_value.to_string()

'Human: 我想去云南旅行，我想知道这个地方有什么好玩的'

### 2.Model
prompt_value 然后传递给模型。在这种情况下，我们的模型是一个 ChatModel，意味着它将输出一个 BaseMessage。

In [20]:
message = model.invoke(prompt_value)
message

AIMessage(content='云南是中国西南部的一个省份，以其多样的地形和丰富的文化而闻名。一些在云南旅行中可以尝试的活动包括：\n\n1. 看日出：在云南的很多地方都有壮丽的日出景观，比如丽江的玉龙雪山和大理的洱海。\n\n2. 登山：云南有许多著名的登山胜地，比如梅里雪山和哈巴雪山，适合喜欢户外探险的旅行者。\n\n3. 探索古镇：云南有许多保存完好的古镇，比如丽江和大理，这些地方充满了浓厚的历史和文化气息。\n\n4. 品尝美食：云南的美食以其多样性和辛辣口味而著称，比如火锅、过桥米线和云南干锅。\n\n5. 欣赏自然风光：云南拥有众多壮丽的自然景观，比如石林、虎跳峡和丘北梯田，都是不容错过的地方。\n\n总的来说，云南是一个充满活力和魅力的省份，无论你是喜欢户外探险还是文化历史，都能在这里找到适合自己的旅行体验。')

### 3.output_parser

最后，我们将模型输出传递给output_parser，它是一个BaseOutputParser，意味着它接受字符串或BaseMessage作为输入。StrOutputParser专门将任何输入简单地转换为字符串。

In [21]:
output_parser.invoke(message)

'云南是中国西南部的一个省份，以其多样的地形和丰富的文化而闻名。一些在云南旅行中可以尝试的活动包括：\n\n1. 看日出：在云南的很多地方都有壮丽的日出景观，比如丽江的玉龙雪山和大理的洱海。\n\n2. 登山：云南有许多著名的登山胜地，比如梅里雪山和哈巴雪山，适合喜欢户外探险的旅行者。\n\n3. 探索古镇：云南有许多保存完好的古镇，比如丽江和大理，这些地方充满了浓厚的历史和文化气息。\n\n4. 品尝美食：云南的美食以其多样性和辛辣口味而著称，比如火锅、过桥米线和云南干锅。\n\n5. 欣赏自然风光：云南拥有众多壮丽的自然景观，比如石林、虎跳峡和丘北梯田，都是不容错过的地方。\n\n总的来说，云南是一个充满活力和魅力的省份，无论你是喜欢户外探险还是文化历史，都能在这里找到适合自己的旅行体验。'

### 4.完整的Pipeline
以下是完整的步骤：
  1. 我们以所需主题的用户输入作为 {"topic": "云南"} 进行传递。
  2. 提示组件接受用户输入，然后使用该主题构建提示后生成 prompt_value。
  3. 模型组件获取生成的提示，并将其传递给 OpenAI LLM 模型进行回答。模型生成的输出是一个 ChatMessage 对象。
  4. 最后，output_parser 组件接收 ChatMessage，并将其转换成 Python 字符串，在调用方法中返回。



## RAG 搜索模板
下一个示例，我们希望运行一个检索增强生成链，在回答问题时添加一些上下文。

In [22]:
from langchain.chat_models import ChatOpenAI # langchain的gpt模型(openai模型的增强)
from langchain.embeddings import OpenAIEmbeddings # OpenAI的embeddings模型
from langchain.prompts import ChatPromptTemplate # 提示词模板
from langchain.vectorstores import DocArrayInMemorySearch # 基于内存的文档数组
from langchain_core.output_parsers import StrOutputParser # 输出解析器
from langchain_core.runnables import RunnableParallel, RunnablePassthrough # 并行运行器, 透传运行器 

# 基于OpenAI的embeddings模型建立索引文档
vectorstore = DocArrayInMemorySearch.from_texts(
  ["小王昨天去了北京，后天回上海","小李昨天去了上海，后天回北京"],
  embedding=OpenAIEmbeddings(
    openai_api_key="",
    openai_api_base=""
  ),
)
retriever = vectorstore.as_retriever()

template = """基于下面的内容，回答下面的问题，如果内容无相关信息，请回复如实回复“我不知道！”。:
{context}

问题: {question}
回答语言:中文
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    )

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever,"question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

print(chain.invoke("小王后天在哪？"))

print(chain.invoke("小李后天在哪？"))

print(chain.invoke("小张后天在哪？"))
  



小王后天在上海。
小李后天在北京。
我不知道！


在这个案例中，链调用的情况如下：

In [23]:
chain = setup_and_retrieval | prompt | model | output_parser

根据这个链调用情况，我们首先可以看到上面的提示模板接受上下文和问题作为要替换在模板中的变量值。在构建提示模板之前，我们希望检索与问题相关的内容并将它们放在模板的**context**变量中。
在一开始，我们已经设置了使用内存存储器的检索器，它可以根据问题来检索文档。这也是一个可运行的组件，并且可以与其他组件链接在一起，但您也可以尝试单独运行它：

In [28]:
retriever.invoke("小王后天在哪?")

[Document(page_content='小王昨天去了北京，后天回上海'),
 Document(page_content='小李昨天去了上海，后天回北京')]

然后我们使用**RunnableParallel**来为预设的**prompt**准备输入值，通过使用检索到的文档条目和原始用户问题，利用检索器进行文档搜索，并使用RunnablePassthrough将用户的问题传递：

In [29]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

回顾一下整个完整的链，如下

In [30]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

完整流程如下：
1. 首先，创建一个包含内容的**RunnableParallel**对象。第一个条目**context**，它将包括检索器获取的文档结果。第二个条目**question**,它将包含用户原始问题。为了传递这个问题，我们使用RunnablePassthrough来复制这个条目。
2. 将上述步骤中的字典传递给**prompt**组件。然后，它会获取用户输入（问题）和检索到的文档（上下文），构建一个提示，并输出一个**PromptValue**。
3. **model**组件接收生成的提示，并将其传递给OpenAI LLM模型进行提问。**model**生成的输出是一个ChatMessage对象。
4. 最后，output_parser 组件接收 ChatMessage，并将其转换成 Python 字符串，在调用方法中返回。
